In [10]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
fn = 'E:/temporary/FURP/week2/CHELSA_pr_1981-2010_V.2.1.nc'  
ds = nc.Dataset(fn)  
df = pd.read_csv('E:/temporary/FURP/week2/estimation_dataset.csv')

In [8]:
tas = ds.variables['tas'][:]
lon = ds.variables['lon'][:]
lat = ds.variables['lat'][:]
month = ds.variables['month'][:]

In [21]:
# Function to find the nearest index
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# Initialize an empty list to store tas values
avg_tas_values_list = []

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    lat_value = row['Lat']
    lon_value = row['Long']
    
    # Find the nearest indices
    lat_idx = find_nearest(lat, lat_value)
    lon_idx = find_nearest(lon, lon_value)
    
    # Extract the tas value for the specific location
    tas_value = tas[:, lat_idx, lon_idx]
    
    # Calculate the average tas value
    avg_tas_value = np.mean(tas_value)
    avg_tas_values_list.append(avg_tas_value)

# Add the tas values to the dataframe
df['Mean Temperature (K)'] = avg_tas_values_list

In [25]:
df.to_csv('E:/temporary/FURP/week2/estimation_dataset.csv', index = True, encoding='utf_8_sig')

In [19]:
# 读取NetCDF文件中的变量
pr = ds.variables['pr'][:]  # 降水量数据
lon = ds.variables['lon'][:]  # 经度数据
lat = ds.variables['lat'][:]  # 纬度数据

In [23]:
# 获取2月的降水量数据（假设索引1是2月）
feb_pr = pr[1, :, :]

In [24]:
# 定义一个函数来找到最接近的索引
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# 初始化一个空列表来存储2月的降水量值
feb_pr_values_list = []

# 遍历数据框的每一行
for index, row in df.iterrows():
    lat_value = row['Lat']
    lon_value = row['Long']
    
    # 找到最近的纬度和经度索引
    lat_idx = find_nearest(lat, lat_value)
    lon_idx = find_nearest(lon, lon_value)
    
    # 提取特定位置2月的降水量值
    pr_value = feb_pr[lat_idx, lon_idx]
    
    # 添加到列表中
    feb_pr_values_list.append(pr_value)

# 将2月的降水量值添加到数据框中
df['Mean Precipitation (2)'] = feb_pr_values_list

In [28]:
!pip install rasterio

In [44]:
import rasterio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 定义 TIFF 文件和 CSV 文件的路径
tif_file = 'E:/temporary/FURP/week2/CHELSA_sfcWind_12_1981-2010_V.2.1.tif'
csv_file = 'E:/temporary/FURP/week2/estimation_dataset.csv'

# 打开 TIFF 文件
with rasterio.open(tif_file) as src:
    # 读取风速数据
    jan_wind_speed = src.read(1)  # 假设风速数据在第一个波段
    lon = src.bounds.left, src.bounds.right
    lat = src.bounds.bottom, src.bounds.top

# 读取 CSV 文件
df = pd.read_csv(csv_file)

# 获取经纬度网格
lon_range = np.linspace(lon[0], lon[1], jan_wind_speed.shape[1])
lat_range = np.linspace(lat[0], lat[1], jan_wind_speed.shape[0])

In [45]:
# 定义一个函数来找到最接近的索引
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# 初始化一个空列表来存储一月的风速值
jan_wind_speed_values_list = []

# 遍历数据框的每一行
for index, row in df.iterrows():
    lat_value = row['Lat']
    lon_value = row['Long']
    
    # 找到最近的纬度和经度索引
    lat_idx = find_nearest(lat_range, lat_value)
    lon_idx = find_nearest(lon_range, lon_value)
    
    # 提取特定位置一月的风速值
    wind_speed_value = jan_wind_speed[lat_idx, lon_idx]
    
    # 添加到列表中
    jan_wind_speed_values_list.append(wind_speed_value)

# 将一月的风速值添加到数据框中
df['Wind Speed (12)'] = jan_wind_speed_values_list


In [46]:
# 保存更新后的数据框到CSV文件
df.to_csv(csv_file, index=True, encoding='utf_8_sig')

In [9]:
import netCDF4 as nc
import pandas as pd
import numpy as np

# 定义NetCDF文件和CSV文件的路径
fn = 'E:/temporary/FURP/week2/CHELSA_tas_1981-2010_V.2.1.nc'
csv_file = 'E:/temporary/FURP/week2/estimation_dataset.csv'

# 打开NetCDF文件
ds = nc.Dataset(fn)

# 读取CSV文件
df = pd.read_csv(csv_file)

# 读取NetCDF文件中的变量
tas = ds.variables['tas'][:]  # 温度数据，假设维度顺序是 (month, lat, lon)
lon = ds.variables['lon'][:]
lat = ds.variables['lat'][:]
month = ds.variables['month'][:]  # 月份数据，假设从1到12表示各个月份

# 定义一个函数来找到最接近的索引
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# 初始化一个字典来存储每个月的平均温度值
monthly_avg_tas_values = {m: [] for m in range(1, 13)}

# 遍历数据框的每一行
for index, row in df.iterrows():
    lat_value = row['Lat']
    lon_value = row['Long']
    
    # 找到最近的纬度和经度索引
    lat_idx = find_nearest(lat, lat_value)
    lon_idx = find_nearest(lon, lon_value)
    
    # 提取特定位置每个月的温度值并存储
    for m in range(1, 13):
        # 假设month的值从1到12，分别表示1到12月
        tas_value = tas[m - 1, lat_idx, lon_idx]
        monthly_avg_tas_values[m].append(tas_value)

# 将每个月的平均温度值添加到数据框中
for m in range(1, 13):
    month_name = f'Mean Temperature ({m})'
    df[month_name] = monthly_avg_tas_values[m]

# 保存更新后的数据框到CSV文件
df.to_csv(csv_file, index=True, encoding='utf_8_sig')

# 关闭NetCDF文件
ds.close()



In [12]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 定义NetCDF文件和CSV文件的路径
fn = 'E:/temporary/FURP/week2/CHELSA_pr_1981-2010_V.2.1.nc'  
csv_file = 'E:/temporary/FURP/week2/estimation_dataset.csv'

# 打开NetCDF文件
ds = nc.Dataset(fn)  

# 读取CSV文件
df = pd.read_csv(csv_file)

# 读取NetCDF文件中的变量
pr = ds.variables['pr'][:]  # 降水量数据，假设维度顺序是 (month, lat, lon)
lon = ds.variables['lon'][:]
lat = ds.variables['lat'][:]

# 定义一个函数来找到最接近的索引
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# 初始化一个字典来存储每个月的降水量值
monthly_pr_values = {m: [] for m in range(1, 13)}

# 遍历数据框的每一行
for index, row in df.iterrows():
    lat_value = row['Lat']
    lon_value = row['Long']
    
    # 找到最近的纬度和经度索引
    lat_idx = find_nearest(lat, lat_value)
    lon_idx = find_nearest(lon, lon_value)
    
    # 提取特定位置每个月的降水量值并存储
    for m in range(1, 13):
        # 假设month的值从1到12，分别表示1到12月
        pr_value = pr[m - 1, lat_idx, lon_idx]
        monthly_pr_values[m].append(pr_value)

# 将每个月的降水量值添加到数据框中
for m in range(1, 13):
    month_name = f'Mean Precipitation ({m})'
    df[month_name] = monthly_pr_values[m]

# 保存更新后的数据框到CSV文件
df.to_csv(csv_file, index=True, encoding='utf_8_sig')

# 关闭NetCDF文件
ds.close()
